<a href="https://colab.research.google.com/github/okane16/data-explorer/blob/main/ibis_sqlglot_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ibis-framework["clickhouse"]
import ibis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 977.6/977.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 66.6 MB/s eta 0:00:00


In [ ]:
from dataclasses import dataclass, fields
from typing import Optional, Union, Any


@dataclass
class Blockchains:
    network: str
    address: str
    decimals: Optional[int]


@dataclass
class CoinRanking:
    id: int
    symbol: str
    name: str
    price: float
    price_btc: float
    volume_24h: float
    volatility: float
    circulating_supply: float
    max_supply: Optional[float]
    percent_change_1h: float
    percent_change_24h: float
    percent_change_7d: float
    market_cap: float
    market_cap_rank: int
    interactions_24h: int
    social_volume_24h: int
    social_dominance: float
    market_dominance: float
    market_dominance_prev: float
    galaxy_score: int
    galaxy_score_previous: int
    alt_rank: int
    alt_rank_previous: int
    sentiment: Optional[int]
    categories: str
    blockchains: list[Blockchains]
    percent_change_30d: float
    last_updated_price: int
    last_updated_price_by: str
    topic: str
    logo: str


In [ ]:
for field in fields(CoinRanking):
    if field.type != primi


Field(name='id',type=<class 'int'>,default=<dataclasses._MISSING_TYPE object at 0x7e6629a3fc40>,default_factory=<dataclasses._MISSING_TYPE object at 0x7e6629a3fc40>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),kw_only=False,_field_type=_FIELD)
Field(name='symbol',type=<class 'str'>,default=<dataclasses._MISSING_TYPE object at 0x7e6629a3fc40>,default_factory=<dataclasses._MISSING_TYPE object at 0x7e6629a3fc40>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),kw_only=False,_field_type=_FIELD)
Field(name='name',type=<class 'str'>,default=<dataclasses._MISSING_TYPE object at 0x7e6629a3fc40>,default_factory=<dataclasses._MISSING_TYPE object at 0x7e6629a3fc40>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),kw_only=False,_field_type=_FIELD)
Field(name='price',type=<class 'float'>,default=<dataclasses._MISSING_TYPE object at 0x7e6629a3fc40>,default_factory=<dataclasses._MISSING_TYPE object at 0x7e6629a3fc40>,init=True,repr=True,

In [ ]:
## Moose could abstract this part away like we do w/ DB client
con = ibis.clickhouse.connect(
    host='v9ksk1d7j4.us-west-2.aws.clickhouse.cloud',
    user='default',
    password='gOaI7o4nC4Qm.',
    secure=True,
    database='default'
)

In [ ]:
schema = con.tables.CoinRanking.schema()

In [ ]:
schema

ibis.Schema {
  id                     !int32
  symbol                 !string
  name                   !string
  price                  float64
  price_btc              float64
  volume_24h             decimal(18, 2)
  volatility             decimal(18, 2)
  circulating_supply     float64
  max_supply             decimal(38, 9)
  percent_change_1h      float64
  percent_change_24h     float64
  percent_change_7d      float64
  market_cap             decimal(18, 2)
  market_cap_rank        int32
  interactions_24h       int32
  social_volume_24h      int32
  social_dominance       float64
  market_dominance       float64
  market_dominance_prev  float64
  galaxy_score           int32
  galaxy_score_previous  int32
  alt_rank               int32
  alt_rank_previous      int32
  sentiment              int32
  categories             string
  blockchains            !array<!struct<network: !string, address: !string, decimals: int64>>
  percent_change_30d     float64
  last_updated_price    

In [ ]:
MyDataModel = ibis.table(schema)

In [ ]:
import sqlglot
from sqlglot import exp
from sqlglot.optimizer import optimize


def convert_to_materialized_view(query: str, view_name: str, target_table_name: str, engine: str) -> str:
    """
    Converts a regular ClickHouse SQL SELECT query into a CREATE MATERIALIZED VIEW statement.
    Ensures all aggregate functions use the `State` suffix.

    Args:
        query (str): Input SQL SELECT query.
        view_name (str): Name of the materialized view.
        engine (str): ClickHouse engine for the target table (e.g., 'AggregatingMergeTree').

    Returns:
        str: SQL for creating the materialized view with an implicit target table.
    """
    # Define suffix mappings for aggregate functions
    suffix_mapping = {
        "MAX": "maxState",
        "AVG": "avgState",
        "SUM": "sumState",
        "MIN": "minState",
        "COUNT": "countState",
    }

    # Parse the query into an AST
    parsed_query = sqlglot.parse_one(query, dialect="clickhouse")
    # Transform aggregate functions to their State equivalents
    def transform_aggregate(node):
        if isinstance(node, exp.AggFunc):
          if node.key != "combinedaggfunc":
            node = exp.CombinedAggFunc(
                this=node.key + "State",
                expressions=[node.args.get("this")],
                parts=(node.key, "State")
            )
            # print(node.error_messages())
          # else:
          #     for arg, val in node.args.items():
          #       print(arg, type(val), val)
        return node

    # Traverse the AST and apply the transformation
    transformed_query = parsed_query.transform(transform_aggregate)

    # # Generate the SQL for the materialized view
    materialized_view_sql = f"""
    CREATE MATERIALIZED VIEW {view_name} TO {target_table_name}
    AS
    {transformed_query.sql(dialect="clickhouse")};
    """
    return materialized_view_sql

    # return target_table_sql.strip() + "\n\n" + materialized_view_sql.strip()

    # # Extract schema information for the target table
    # select_expressions = transformed_query.find(exp.Select).expressions
    # schema = []
    # for expression in select_expressions:
    #     if isinstance(expression, exp.Alias):
    #         alias = expression.alias
    #         if isinstance(expression.this, exp.CombinedAggFunc):
    #             func_name = expression.this.name
    #             schema.append(f"{alias} AggregateFunction({func_name}, Float64)")
    #         else:
    #             schema.append(f"{alias} String")
    #     else:
    #         schema.append(f"{expression.sql()} String")  # Default to String

    # # Generate the SQL for the target table
    # target_table_sql = f"""
    # CREATE TABLE {view_name} (
    #     {', '.join(schema)}
    # )
    # ENGINE = {engine}()
    # ORDER BY {', '.join([exp.sql() for exp in transformed_query.find(exp.Group).expressions])};
    # """


# Example Usage
# query = """
# SELECT
#     category AS category,
#     MAX(price) AS max_price,
#     AVG(price) AS avg_price
# FROM sales
# GROUP BY category;
# """
query = """
SELECT
  t2.categories AS categories,
  max(t2.galaxy_score) AS max_galaxy_score,
  avg(t2.price) AS avg_price,
  count(t2.symbol) AS btc_count,
  quantile(0.75)(t2.price) AS q75_galaxy_score,
FROM (
  SELECT
    t1.id AS id,
    t1.symbol AS symbol,
    t1.name AS name,
    t1.price AS price,
    t1.galaxy_score AS galaxy_score
  FROM (
    SELECT *
    FROM CoinRanking AS t0
    WHERE isNotNull(t0.categories)
  ) AS t1
) AS t2
GROUP BY
  t2.categories
"""

view_name = "mv_sales_by_category"
engine = "AggregatingMergeTree"
table_name = "sales"

result_sql = convert_to_materialized_view(query, view_name, table_name, engine)
print(result_sql)


    CREATE MATERIALIZED VIEW mv_sales_by_category TO sales
    AS
    SELECT t2.categories AS categories, maxState(t2.galaxy_score) AS max_galaxy_score, avgState(t2.price) AS avg_price, countState(t2.symbol) AS btc_count, quantileState(t2.price) AS q75_galaxy_score FROM (SELECT t1.id AS id, t1.symbol AS symbol, t1.name AS name, t1.price AS price, t1.galaxy_score AS galaxy_score FROM (SELECT * FROM CoinRanking AS t0 WHERE isNotNull(t0.categories)) AS t1) AS t2 GROUP BY t2.categories;
    


In [ ]:
import ibis.selectors as s
import re

def sanitize_sql_query(query: str) -> str:
    # Use a regular expression to find quoted column names and remove the quotes
    sanitized_query = re.sub(r'"(\w+)"', r'\1', query)
    return sanitized_query



cr = con.tables.CoinRanking ## Replace w/ Moose Data Model, or can possibly have the table as part of the Data Model primtiive...?

## Convert categories column, a string of comma separated category names, to a list, then unnest the list
unnest_categories = cr.filter(cr.categories.notnull()).mutate(
    categories = cr.categories.split(',').unnest()
)

## Now group by the categories and aggregate over each category
expr = unnest_categories.group_by(unnest_categories.categories).aggregate(unnest_categories.galaxy_score.max(), unnest_categories.price.mean())

## FROM nested queries -> chained operations, more intuitive and easier to implement than raw SQL

## Convert to SQL
print(sanitize_sql_query(ibis.to_sql(expr, dialect="clickhouse")))

con.execute(expr)



SELECT
  t2.categories AS categories,
  MAX(t2.galaxy_score) AS "Max(galaxy_score)",
  AVG(t2.price) AS "Mean(price)"
FROM (
  SELECT
    t1.id AS id,
    t1.symbol AS symbol,
    t1.name AS name,
    t1.price AS price,
    t1.price_btc AS price_btc,
    t1.volume_24h AS volume_24h,
    t1.volatility AS volatility,
    t1.circulating_supply AS circulating_supply,
    t1.max_supply AS max_supply,
    t1.percent_change_1h AS percent_change_1h,
    t1.percent_change_24h AS percent_change_24h,
    t1.percent_change_7d AS percent_change_7d,
    t1.market_cap AS market_cap,
    t1.market_cap_rank AS market_cap_rank,
    t1.interactions_24h AS interactions_24h,
    t1.social_volume_24h AS social_volume_24h,
    t1.social_dominance AS social_dominance,
    t1.market_dominance AS market_dominance,
    t1.market_dominance_prev AS market_dominance_prev,
    t1.galaxy_score AS galaxy_score,
    t1.galaxy_score_previous AS galaxy_score_previous,
    t1.alt_rank AS alt_rank,
    t1.alt_rank_previous

,categories,Max(galaxy_score),Mean(price)
0,depin,78,5.054894
1,runes,88,0.154775
2,socialfi,69,0.378718
3,fan,74,0.737444
4,layer-1,78,556.743215
5,sports,64,0.896169
6,gaming,76,1.040270
7,meme,86,1.124287
8,dao,95,55.925087
9,gambling,68,0.078818
